In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, _tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('drug200_1722406166375.csv')

# Preprocess the data
X = data[['Age', 'Sex', 'BP', 'Cholesterol']]
y = data['Drug']

X = pd.get_dummies(X, columns=['Sex', 'BP', 'Cholesterol'], drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CustomDecisionTreeClassifier
class CustomDecisionTreeClassifier(DecisionTreeClassifier):
    def __init__(self, criterion, min_purity, **kwargs):
        super().__init__(criterion=criterion, **kwargs)
        self.min_purity = min_purity

    def fit(self, X, y):
        super().fit(X, y)
        self._prune_tree()

    def _prune_tree(self):
        if not hasattr(self, 'tree_'):
            return

        def prune_node(node):
            if self.tree_.feature[node] == _tree.TREE_UNDEFINED:
                return

            left_child = self.tree_.children_left[node]
            right_child = self.tree_.children_right[node]

            if left_child == right_child:
                return

            # Recursively prune children
            prune_node(left_child)
            prune_node(right_child)

            # Check if the node should be pruned
            if self._should_prune(node, left_child, right_child):
                self.tree_.feature[node] = _tree.TREE_UNDEFINED
                self.tree_.threshold[node] = -2
                self.tree_.value[node] = self.tree_.value[left_child] + self.tree_.value[right_child]
                self.tree_.children_left[node] = self.tree_.children_right[node] = _tree.TREE_LEAF

        prune_node(0)  # Start pruning from the root node

    def _should_prune(self, node, left_child, right_child):
        left_purity = self._calculate_purity(left_child)
        right_purity = self._calculate_purity(right_child)
        return left_purity >= self.min_purity and right_purity >= self.min_purity

    def _calculate_purity(self, node):
        if self.tree_.feature[node] == _tree.TREE_LEAF:
            total_samples = np.sum(self.tree_.value[node])
            max_class_samples = np.max(self.tree_.value[node])
            return max_class_samples / total_samples if total_samples > 0 else 0
        return 0

# Instantiate and fit the classifier
clf = CustomDecisionTreeClassifier(criterion='entropy', min_purity=1.0)
clf.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(export_text(clf, feature_names=list(X.columns)))



Accuracy: 0.575
|--- BP_NORMAL <= 0.50
|   |--- BP_LOW <= 0.50
|   |   |--- Age <= 50.50
|   |   |   |--- Age <= 46.50
|   |   |   |   |--- Age <= 19.00
|   |   |   |   |   |--- class: drugY
|   |   |   |   |--- Age >  19.00
|   |   |   |   |   |--- Sex_M <= 0.50
|   |   |   |   |   |   |--- Age <= 20.50
|   |   |   |   |   |   |   |--- class: drugA
|   |   |   |   |   |   |--- Age >  20.50
|   |   |   |   |   |   |   |--- Age <= 25.00
|   |   |   |   |   |   |   |   |--- class: drugY
|   |   |   |   |   |   |   |--- Age >  25.00
|   |   |   |   |   |   |   |   |--- Age <= 36.50
|   |   |   |   |   |   |   |   |   |--- Age <= 33.00
|   |   |   |   |   |   |   |   |   |   |--- Cholesterol_NORMAL <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: drugY
|   |   |   |   |   |   |   |   |   |   |--- Cholesterol_NORMAL >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 3
|   |   |   |   |   |   |   |   |   |--- Age >  33.00
|   |   |   |   |   |  